In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import pickle
import re
from tashaphyne import normalize
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()
from collections import Counter

In [2]:
predict_price = keras.models.load_model('numerical.h5',compile=False)

In [3]:
predict_price.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                30000     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 30,051
Trainable params: 30,051
Non-trainable params: 0
_________________________________________________________________


In [4]:
sentiment = pickle.load(open('sentiment.pkl','rb'))
vectorizer = pickle.load(open('vectorizer.pkl','rb'))
all_tweets = pickle.load(open('all_tweets.pkl','rb'))

In [5]:
all_tweets.dtypes

company_name           object
Date                   object
likes                  object
retweet                object
username               object
number of followers    object
content                object
url                    object
dtype: object

In [6]:
all_tweets = all_tweets[all_tweets['Date'] != 'Date']

In [7]:
all_tweets['Date'] = pd.to_datetime(all_tweets['Date'])

In [8]:
all_tweets['Date'] = all_tweets['Date'].dt.date
all_tweets['Date'] = pd.to_datetime(all_tweets['Date'])

In [9]:
all_tweets.dtypes

company_name                   object
Date                   datetime64[ns]
likes                          object
retweet                        object
username                       object
number of followers            object
content                        object
url                            object
dtype: object

In [10]:
all_tweets.head()

,company_name,Date,likes,retweet,username,number of followers,content,url
0,Aldrees,2022-09-21,11,2,ali21986,31,@gorgeous4ew اعلان شركة الدريس https://t.co/u5...,https://twitter.com/ali21986/status/1572724914...
1,Aldrees,2022-09-21,1,0,AshrafEejazz1,1031,@toa5968 منشأة تجارية وخدمية رائعة ومميزة وراق...,https://twitter.com/AshrafEejazz1/status/15727...
2,Aldrees,2022-09-21,0,0,SaadAlawwad6,63,@f_alabdulkarim شركة الدريس 🇸🇦 @aldreescompany...,https://twitter.com/SaadAlawwad6/status/157271...
3,Aldrees,2022-09-21,1,0,rageb_14,581,وف الاخر مقدميش غير تيد لاسو(بمناسبة أنه تريند...,https://twitter.com/rageb_14/status/1572700923...
4,Aldrees,2022-09-21,2,0,FUADALABADI1,1084,@SALEH16428337 @3o6KdiTDbJpROvs لا عيني هذا ال...,https://twitter.com/FUADALABADI1/status/157270...


In [11]:
def clean_text(text):
    text = re.sub(r'[^ء-ي]',' ',text)
    text = normalize.normalize_searchtext(text)
    text = ' '.join([st.stem(word) for word in text.split()])# if word not in stop_words])
    return text

In [12]:
all_tweets.dtypes

company_name                   object
Date                   datetime64[ns]
likes                          object
retweet                        object
username                       object
number of followers            object
content                        object
url                            object
dtype: object

In [13]:
def get_sentiment(date):
    filtered_tweets = all_tweets[all_tweets['Date'].between(date,date)]
    filtered_tweets = filtered_tweets['content'].apply(clean_text)
    vectorized_text = vectorizer.transform(filtered_tweets)
    x = sentiment.predict(vectorized_text)
    return list(Counter(x).keys())[0]

In [14]:
res = get_sentiment('2022-09-21')

In [15]:
res

1

In [16]:
df = pd.read_excel('forex data.xlsx')
df.columns = ['date','open','high','low','close','vol']
open_df = df['open']
lst = []
for row in range(100,df.shape[0]):#
    #print(df.loc[row,'open'])
    lst.append(open_df[row-100:row].values)
forcasting_table = pd.DataFrame(lst)

In [17]:
forcasting_table

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,97.20,98.50,102.60,104.60,104.00,105.40,108.4,107.80,108.20,106.20,...,113.40,117.60,117.80,119.40,120.00,119.00,118.20,120.00,123.80,125.00
1,98.50,102.60,104.60,104.00,105.40,108.40,107.8,108.20,106.20,104.40,...,117.60,117.80,119.40,120.00,119.00,118.20,120.00,123.80,125.00,124.40
2,102.60,104.60,104.00,105.40,108.40,107.80,108.2,106.20,104.40,102.80,...,117.80,119.40,120.00,119.00,118.20,120.00,123.80,125.00,124.40,124.00
3,104.60,104.00,105.40,108.40,107.80,108.20,106.2,104.40,102.80,103.60,...,119.40,120.00,119.00,118.20,120.00,123.80,125.00,124.40,124.00,126.00
4,104.00,105.40,108.40,107.80,108.20,106.20,104.4,102.80,103.60,106.00,...,120.00,119.00,118.20,120.00,123.80,125.00,124.40,124.00,126.00,125.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,62.50,62.00,62.75,62.50,65.25,64.75,64.5,66.50,67.00,66.00,...,68.75,67.75,68.00,71.00,73.00,76.50,76.25,73.50,74.50,74.00
2387,62.00,62.75,62.50,65.25,64.75,64.50,66.5,67.00,66.00,65.50,...,67.75,68.00,71.00,73.00,76.50,76.25,73.50,74.50,74.00,73.25
2388,62.75,62.50,65.25,64.75,64.50,66.50,67.0,66.00,65.50,65.75,...,68.00,71.00,73.00,76.50,76.25,73.50,74.50,74.00,73.25,75.50
2389,62.50,65.25,64.75,64.50,66.50,67.00,66.0,65.50,65.75,65.25,...,71.00,73.00,76.50,76.25,73.50,74.50,74.00,73.25,75.50,79.00


In [18]:
forcasting_table['date'] = df.iloc[:-100,0]

In [19]:
forcasting_table['date'] = pd.to_datetime(forcasting_table['date'])

In [20]:
forcasting_table

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,date
0,97.20,98.50,102.60,104.60,104.00,105.40,108.4,107.80,108.20,106.20,...,117.60,117.80,119.40,120.00,119.00,118.20,120.00,123.80,125.00,2022-09-08
1,98.50,102.60,104.60,104.00,105.40,108.40,107.8,108.20,106.20,104.40,...,117.80,119.40,120.00,119.00,118.20,120.00,123.80,125.00,124.40,2022-09-07
2,102.60,104.60,104.00,105.40,108.40,107.80,108.2,106.20,104.40,102.80,...,119.40,120.00,119.00,118.20,120.00,123.80,125.00,124.40,124.00,2022-09-06
3,104.60,104.00,105.40,108.40,107.80,108.20,106.2,104.40,102.80,103.60,...,120.00,119.00,118.20,120.00,123.80,125.00,124.40,124.00,126.00,2022-09-05
4,104.00,105.40,108.40,107.80,108.20,106.20,104.4,102.80,103.60,106.00,...,119.00,118.20,120.00,123.80,125.00,124.40,124.00,126.00,125.00,2022-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,62.50,62.00,62.75,62.50,65.25,64.75,64.5,66.50,67.00,66.00,...,67.75,68.00,71.00,73.00,76.50,76.25,73.50,74.50,74.00,2013-02-12
2387,62.00,62.75,62.50,65.25,64.75,64.50,66.5,67.00,66.00,65.50,...,68.00,71.00,73.00,76.50,76.25,73.50,74.50,74.00,73.25,2013-02-11
2388,62.75,62.50,65.25,64.75,64.50,66.50,67.0,66.00,65.50,65.75,...,71.00,73.00,76.50,76.25,73.50,74.50,74.00,73.25,75.50,2013-02-10
2389,62.50,65.25,64.75,64.50,66.50,67.00,66.0,65.50,65.75,65.25,...,73.00,76.50,76.25,73.50,74.50,74.00,73.25,75.50,79.00,2013-02-09


In [21]:
def get_price(date):
    row = forcasting_table[forcasting_table['date'].between(date,date)].iloc[:,:-2]
    row = row.values.reshape((row.shape[0],1, row.shape[1]))
    predicted_price = predict_price.predict(row)
    return predicted_price[0]

In [22]:
res = get_price('2022-09-08')

1/1 [==============================] - 0s 285ms/step


In [23]:
def predict_new_price(date):
    polarity = get_sentiment(date)
    price = get_price(date)
    if polarity == 0:
        #negative
        price = price - price * 0.1
    elif polarity == 1:
        #positive
        price = price + price * 0.1
    else:
        #neutral
        price = price
    return price

In [24]:
final_price = predict_new_price('2022-09-08')

1/1 [==============================] - 0s 49ms/step


In [25]:
final_price

array([132.95126], dtype=float32)